# ISOLATION Forest

In [490]:
d = [[1,1,2],[3,2,4],[2,1,2],[3,1,2],[7,8,9],[2,1,2],[11,1,11]]

In [491]:
def split_data(data, col):
    try:
        feature_data = [line[col] for line in data]
    except TypeError as e:
        raise TypeError("the type is not fit de algo")
    feature_min_value = min(feature_data)
    feature_max_value = max(feature_data)
    random_value = feature_min_value + np.random.rand(1) * (feature_max_value - feature_min_value)
    left_data = []
    right_data = []
    for line in data:
        if line[col] >= random_value:
            right_data.append(line)
        else:
            left_data.append(line)
    return left_data, right_data, col, random_value


def sample_data(data, ratio):
    if isinstance(data, list):
        m, n = len(data), len(data[0])
    elif isinstance(data, np.ndarray):
        m, n = data.shape
    
    if 0 < ratio < 1:
        sub_m = int(ratio * m)
    elif isinstance(ratio, int):
        sub_m = min(ratio, m)
    
    new_data = []
    random_indx = np.random.permutation(m)[: sub_m]
    for idx in random_indx:
        new_data.append(data[idx])
    return new_data, sub_m, n
   
# split_data(d)
# random_data(d, ratio=3)

In [492]:
def get_max_deep(num):
    return np.ceil(np.log2(num))

def cost(num_items):
    if num_items > 2:
    # 二叉搜索树的平均路径长度。0.5772156649:欧拉常数
        re =  2 * ((np.log2(num_items - 1) + 0.5772156649) - (num_items - 1) / num_items)
        return re
    elif num_items == 2:
        return 1
    else:
        return 0

In [493]:
def createTree(data, num_col, max_deep, cur_deep = 0):
    m, n = np.mat(data).shape
    if cur_deep >= max_deep or m <= 1:  # 这个地方的 1 需要修稿的
        return m

    # 随机选择 一个特征进行数据划分
    col = np.random.randint(num_col)
    lSet, rSet, col, random_value = split_data(data, col)
    retTree = {}
    retTree['spInd'] = col
    retTree['spVal'] = random_value
    retTree['left'] = createTree(lSet, num_col, max_deep, cur_deep + 1)
    retTree['right'] = createTree(rSet, num_col, max_deep, cur_deep + 1)
    return retTree

In [495]:
def isolation_forest(data, ratio, num_trees=10):
    Trees = []
    sub_ms = []
    for i in range(10):
        random_data, sub_m, num_feature = sample_data(data, ratio)
        max_deep = get_max_deep(sub_m)
        ITree = createTree(random_data, num_feature, max_deep)
        Trees.append(ITree)
        sub_ms.append(sub_m)
    return Trees, sub_ms
    
#trees,sub_ms = isolation_forest(d,5)
#trees

def predict_score(x, ITrees, data_num):
    predictions = []
    for itree in ITrees:
        predict = get_length(itree, x)
        predictions.append(predict)
    return np.power(2, -np.sum(predictions) /len(predictions) / cost(data_num))  

trees = isolation_forest(np.array(d), ratio=0.9)
predict_score([100,200,300],trees,5)

TypeError: list indices must be integers or slices, not str

In [487]:
def get_length(tree, x): 
    split_value = tree['spVal']
    split_ind = tree['spInd']
    
    if x[split_ind] > split_value:
        if isinstance(tree['right'], dict):
            mdeep = 1 + get_length(tree['right'], x)
        else:
            mdeep = cost(tree['right']) # 注意这里的平滑系数。
    else: 
        if isinstance(tree['left'],dict):
            mdeep = 1 + get_length(tree['left'], x)
        else:
            mdeep = cost(tree['left']) # 当 2为 当样本数量很少时。
    return mdeep

get_length(mytree,x = [0,0,1])

TypeError: 'type' object is not subscriptable

In [424]:
def predict_score(x, ITrees, data_num):
    predictions = []
    for itree in ITrees:
        predict = get_length(itree, x)
        predictions.append(predict)
    return np.power(2, -np.sum(predictions) /len(predictions) / cost(data_num))  

predict_score([100,200,300],trees,5)

In [ ]:
# 计算树的 deep 及 叶子树

In [378]:
def tree_leafs(mytree):
    num_feature = 0
    for key in mytree.keys():
        if isinstance(mytree[key], dict):
            num_feature += tree_leafs(mytree[key])
        else:
            num_feature += 1 # mytree[key]
    return num_feature
tree_leafs(mytree)

16

In [379]:
def tree_deep(mytree):
    max_deep = 0
    for key in mytree.keys():
        if isinstance(mytree[key], dict):
            temp_deep = tree_deep(mytree[key]) + 1
        else:
            temp_deep = 1   
        max_deep = max(max_deep, temp_deep)
    return max_deep
tree_deep(mytree)

3

# ID3

In [138]:
# 多叉树 ，且每个特征只使用一次，只适合 类型较少的 分类型数据。

import numpy as np
def createDataSet():
    dataSet = [[1, 1,2, 'yes'],
               [1, 1,3,'yes'],
               [2, 0,1,'no'],
               [0, 1,3,'no'],
               [0, 1,3,'no'],
               [2, 2,1,'yes']   
              ]
              #[1,1,'maybe']]
    labels = ['no surfacing','lk','flippers']
    return dataSet, labels

data,label = createDataSet()

In [139]:
def cal(data)->"input:最后一列带标签的数据，output:entropy": 
    cout_dic ={}
    if isinstance(data,list):
        m = len(data)
    elif isinstance(data,np.ndarray):  # 这里需要注意哦。 numpy.ndarray
        m = data.shape[0]
    else:
        raise ValueError("The type of the data must be LIST or ARRAY")
    for i in range(m):
        temp = data[i][-1]
        cout_dic.setdefault(temp,0)
        cout_dic[temp] +=1
    cout_dic = {k:-(v/m)*(np.log2(v/m)) for k,v in cout_dic.items()}
    entrop = sum(cout_dic.values())
    return entrop

cal(np.array(data))

1.0

In [140]:
def gini(data):
    cout_dic ={}
    m = len(data)
    for i in range(m):
        temp = data[i][-1]
        cout_dic.setdefault(temp,0)
        cout_dic[temp] +=1
    cout_dic = {k:(v/m)**2 for k,v in cout_dic.items()}
    gini = 1- sum(cout_dic.values())
    return gini,cout_dic
gini(data)

(0.5, {'no': 0.25, 'yes': 0.25})

In [141]:
def split_feature(data,k=0):
    m = len(data)
    #n = len(data[0])
    data = np.array(data)
    t = np.unique(data[:,k])
    splitdata={}
    for i in t:
        for j in range(m):
            if data[j,k]==i:
                splitdata.setdefault(i,[])
                splitdata[i].append(data[j])
    return splitdata

In [142]:
"""
new_entrop=0.
for key,value in xx.items():
    length =len(value)
    new_entrop +=length/5 *cal(value)
    #new_entrop +=cal(value)
    print(new_entrop)
"""

'\nnew_entrop=0.\nfor key,value in xx.items():\n    length =len(value)\n    new_entrop +=length/5 *cal(value)\n    #new_entrop +=cal(value)\n    print(new_entrop)\n'

In [ ]:
（1）计算未 划分前的entropy
（2）根据每一个 特征的 值 进行划分，计算 entropy的改变量，选择 改变量最大的 特征和 特征值。
（3）然后 递归的 对数据进行划分，只到满足特定的条件
（4）计算 树的 最大深度 和枝叶数目

In [143]:
# 选择最优的分裂的枝叶。
def chose_best_feature(data):
    if isinstance(data, list):
        old_entrop = cal(data)
        n = len(data[0]) - 1
        entrop_change = {}
        len_data = len(data)
    else:
        raise TypeError("The type of the data must be List")
    
    for i in range(n):
        temp = split_feature(data, k=i)
        new_entrop = 0.
        for key, value in temp.items():
            length = len(value)
            new_entrop += length / len_data * cal(value) #这里注意用法。 数据集划分之后 标签的 信息熵。
        entrop_change.setdefault(i, 0)
        entrop_change[i] = old_entrop - new_entrop
    sort_entrop_change = sorted(entrop_change.items(), key=lambda row: row[1], reverse=True)
    best_feature =sort_entrop_change[0][0]
    
    return sort_entrop_change, best_feature

In [144]:
def majorityCnt(classList):
    classCount = {}
    for vote in classList:
        classCount.setDefault(vote,0)
        classCount[vote] +=1
    sort = sorted(classCount.items(),key=lambda row:row[1],reverse=True)
    return sort[0][0]

In [145]:
def split_feature_x(data,axis,value):
    splitdata=[]
    for vect in data:
        if vect[axis]==value:
            temp = vect[:axis]
            temp.extend(vect[axis+1:])
            splitdata.append(temp)
    return splitdata
split_feature_x(data,0,1)

[[1, 2, 'yes'], [1, 3, 'yes']]

In [146]:
def createTree(dataset,label):
    labels = label[:]
    classList = [example[-1] for example in dataset]    # 因为后面对数据进行了切分，因此 dataset只是一个子集。
    if classList.count(classList[0]) == len(classList): # 当一组数据的标签都完全一致是 则 停止分裂。
        return classList[0]
    if len(dataset[0])==1:                              # 没有特征可供划分时 则 停止划分， 么有往复的使用特征。
        return majorityCnt(classList)                   # 这里使用的 投票法。
    
    _, bestFeat = chose_best_feature(dataset)
    bestFeatLabel = labels[bestFeat]                    # labels 中存储的 是特制的 字符串名称 不是 取值，因此转换。
    
    myTree ={bestFeatLabel:{}}
    del labels[bestFeat]  # 消除已经使用过的特征。
    
    featValues = [example[bestFeat] for example in dataset]
    uniqueVals = set(featValues)
    for value in uniqueVals:
        subLabels = labels[:]
        myTree[bestFeatLabel][value]=createTree(split_feature_x(      # 根据特征的 每个类别 构建 多差树。
        dataset, bestFeat, value), subLabels)
    return myTree

In [147]:
data,label = createDataSet()
t = createTree(data,label)
print(t)
fir =[i for i in t.keys()][0]
fir

{'no surfacing': {0: 'no', 1: 'yes', 2: {'lk': {0: 'no', 2: 'yes'}}}}


'no surfacing'

In [94]:
firstStr = [i for i in t.keys()][0]
print(firstStr)
secondDict = t[firstStr]
print(secondDict)
featIndex = labels.index(firstStr) # 在 Label 中 firstStr所在的位置 list.index("xx")
print(featIndex)
print(secondDict.keys())

no surfacing
{0: 'no', 1: 'yes', 2: 'no'}
0
dict_keys([0, 1, 2])


In [149]:
def classify(inputTree,featLabels,testVec):
    firstStr = [i for i in inputTree.keys()][0]
    secondDict = inputTree[firstStr]
    featIndex = featLabels.index(firstStr)
    
    for key in secondDict.keys():
        if testVec[featIndex]==key:
            #if type(secondDict[key]).__name__=='dict':
            if isinstance(secondDict[key],dict):
                classLabel = classify(secondDict[key],featLabels,testVec)
            else:
                classLabel = secondDict[key]
    return classLabel

In [150]:
fs = [i for i in mytree.keys()][0]
sd = mytree[fs]
sd
fi = labels.index(fs)
fi

0

In [151]:
data,labels = createDataSet()
feature_label = labels[:]
mytree = createTree(data,labels)
#classify(mytree,feature_label,[1,1])
classify(mytree,labels,[1,0])

'yes'

In [152]:
mytree#.keys()

{'no surfacing': {0: 'no', 1: 'yes', 2: {'lk': {0: 'no', 2: 'yes'}}}}

In [153]:
for i in mytree.keys():
    print(i)
    print(mytree[i])

no surfacing
{0: 'no', 1: 'yes', 2: {'lk': {0: 'no', 2: 'yes'}}}


In [154]:
def storetree(putTree,filename):
    import pickle
    fw = open(filename,'wb')
    pickle.dump(putTree,fw)
    fw.close()
    
def loadtree(filename):
    import pickle
    fr = open(filename,'rb')
    return pickle.load(fr)

storetree(createTree,'./filetree.txt')

m = loadtree('./filetree.txt')

data,labels = createDataSet()
m(data,labels)

{'no surfacing': {0: 'no', 1: 'yes', 2: {'lk': {0: 'no', 2: 'yes'}}}}

In [167]:
x = list(mytree.keys())[0]
x
se = mytree[x]
se.keys()
se[0]

'no'

In [170]:
def tree_leafs(mytree):
    first_node = list(mytree.keys())[0]
    num_leafs = 0
    sencond_tree = mytree[first_node]
    for key in sencond_tree.keys():
        if isinstance(sencond_tree[key],dict):
            num_leafs += tree_leafs(sencond_tree[key])
        else:
            num_leafs +=1 
    return num_leafs

In [179]:
def tree_deep(mytree):
    first_node = list(mytree.keys())[0]
    maxdeep = 0
    second_tree = mytree[first_node]
    for key in second_tree.keys():
        if isinstance(second_tree[key],dict):
            temp_deep = tree_deep(second_tree[key]) +1
        else:
            temp_deep = 1
        maxdeep = max(maxdeep,temp_deep)
    return maxdeep
     
tree_deep(mytree)

2

# GITHUB上的

In [434]:
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
import pandas as pd


def _h(i):
    return np.log(i) + 0.5772156649 


def _c(n):
    if n > 2:
        h = _h(n-1)
        return 2*h - 2*(n - 1)/n
    if n == 2:
        return 1
    else:
        return 0


def _anomaly_score(score, n_samples):

    score = -score/_c(n_samples)

    return 2**score


def _split_data(X):
    ''' split the data in the left and right nodes ''' 
    n_samples, n_columns = X.shape
    n_features = n_columns - 1
    m = M = 0
    while m == M:
        feature_id = np.random.randint(low=0, high=n_features)
        feature = X[:, feature_id]
        m = feature.min()
        M = feature.max()
        #print(m, M, feature_id, X.shape)

    split_value = np.random.uniform(m, M, 1)
    left_X = X[feature <= split_value]
    right_X = X[feature > split_value]
    return left_X, right_X, feature_id, split_value


def iTree(X, add_index=False, max_depth = np.inf):            
    ''' construct an isolation tree and returns the number of step required
    to isolate an element. A column of index is added to the input matrix X if  
    add_index=True. This column is required in the algorithm. ''' 

    n_split = {} 
    def iterate(X, count = 0):

        n_samples, n_columns = X.shape
        n_features = n_columns - 1

        if count > max_depth:
            for index in X[:,-1]:
                n_split[index] = count
            return

        if n_samples == 1:
            index = X[0, n_columns-1]
            n_split[index] = count
            return 
        else:
            lX, rX, feature_id, split_value = _split_data(X)
            # Uncomment the print to visualize a draft of 
            # the construction of the tree
            #print(lX[:,-1], rX[:,-1], feature_id, split_value, n_split)
            n_samples_lX, _ = lX.shape
            n_samples_rX, _ = rX.shape
            if n_samples_lX > 0:
                iterate(lX, count+1)
            if n_samples_rX >0:
                iterate(rX, count+1)

    if add_index:
        n_samples, _ = X.shape
        X = np.c_[X, range(n_samples)]

    iterate(X)
    return n_split


class iForest():
    ''' Class to construct the isolation forest.

    -n_estimators: is the number of trees in the forest,

    -sample_size: is the bootstrap parameter used during the construction
    of the forest,

    -add_index: adds a column of index to the matrix X. This is required and 
    add_index can be set to False only if the last column of X contains 
    already indeces.

    -max_depth: is the maximum depth of each tree
    '''
    def __init__(self, n_estimators=20, sample_size=None, add_index = True, 
                 max_depth = 100):
        self.n_estimators = n_estimators
        self.sample_size = sample_size
        self.add_index = add_index
        self.max_depth = max_depth
        return 

    def fit(self, X):
        n_samples, n_features = X.shape
        if self.sample_size == None:
            self.sample_size = int(n_samples/2)

        if self.add_index:
            X = np.c_[X, range(n_samples)]


        trees = [iTree(X[np.random.choice(n_samples, self.sample_size,replace=False)]
                       ,max_depth=self.max_depth) for i in range(self.n_estimators)]

        self.path_length_ = {k:None for k in range(n_samples)}
        for k in self.path_length_.keys():
            self.path_length_[k] = np.array([tree[k] for tree in trees  if k in tree])
        
        self.path_length_ = np.array([self.path_length_[k].mean() for k in 
                                      self.path_length_.keys()])
        self.anomaly_score_ = _anomaly_score(self.path_length_, self.sample_size)
        return self

/Users/luokui/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/luokui/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


## cart tree 

In [3]:
import numpy as np
from sklearn.datasets import load_iris
from collections import Counter

In [468]:
class mytree:
    def __init__(self, feature=-1, value=None, left_tree=None, right_tree=None, label_class=None):
        self.feature = feature
        self.value = value
        self.left_tree = left_tree
        self.right_tree = right_tree
        self.label_class = label_class

class mycarttree:       
    def __init__(self, min_leafs):
        self.min_leafs = min_leafs
    
    def calculate_gini(self, label):
        temp_dicts = {}
        for temp_label in label:
            temp_label = temp_label[0]
            temp_dicts.setdefault(temp_label, 0)
            temp_dicts[temp_label] += 1
            
        temp_count = np.array([i for i in temp_dicts.values()])
        sum_values = np.sum(temp_count)
        gini_value = 1 - np.sum(np.power(temp_count/sum_values, 2))
        return gini_value
    
    def split_data(self, data, feature, value, label):
        if not label.shape:
            raise TypeError("label muste be two dim")
        m, n = data.shape
        condition_left = data[:, feature] <= value
        condition_right = data[:, feature] > value
        
        left_data = data[condition_left]
        right_data = data[condition_right]
        left_label = label[condition_left]
        right_label = label[condition_right]

        split_gini = (left_label.shape[0]/m) * self.calculate_gini(left_label) + (right_label.shape[0]/m) * self.calculate_gini(right_label)
        return left_data, left_label, right_data, right_label, split_gini
    
    def find_best_split(self, data, label):
        m, n = data.shape
        
        best_feature = None
        best_feature_value = None
        best_left_data = None
        best_right_data = None
        best_left_label = None
        best_right_label = None
        
        max_ginigain = 0
        init_gini = self.calculate_gini(label)
        
        for feature_index in range(n):
            feature_values = set(data[:, feature_index])
            for feature_value in feature_values:
                left_data, left_label, right_data, right_label, split_gini = self.split_data(data, feature_index, feature_value, label)
                temp_ginigain = init_gini - split_gini
                if temp_ginigain >  max_ginigain and len(left_data)>0 and len(right_data)>0:
                    max_ginigain = temp_ginigain
                    best_feature = feature_index
                    best_feature_value = feature_value
                    best_left_data = left_data
                    best_right_data = right_data
                    best_left_label = left_label
                    best_right_label = right_label
        return best_feature, best_feature_value, best_left_data, best_right_data, best_left_label, best_right_label
       
    def get_result(self, label):
        return Counter([i[0] for i in label]).most_common()[0][0]
        
    def fit(self, data, label):
        if len(data) == 0:
            return mytree()
        elif len(np.unique(label))==1:
            return mytree(label_class = self.get_result(label))
        
        best_feature, best_feature_value, best_left_data, best_right_data, best_left_label, best_right_label = self.find_best_split(data, label)
        
        if best_feature:
            right = self.fit(best_right_data, best_right_label)
            left = self.fit(best_left_data, best_left_label)
            
            mytrees.feature = best_feature
            mytrees.value = best_feature_value
            mytrees.left_tree = left
            mytrees.right_tree = right
            return mytree(feature=best_feature, value=best_feature_value, left_tree=left, right_tree=right)
        else:
            return mytree(label_class = self.get_result(label))
            
    #@staticmethod
    @classmethod
    def predict_line(cls, model, line, high=0):
        #m, n = line.shape
        #if m != 1:
         #   raise TypeError("xx")
        # 这里需要注意-> 这里只会返回一个值，而不是多个值。
        if model.label_class is not None:
            # print(high)
            return model.label_class
        split_feature = model.feature
        split_value = model.value
        
        temp_value = line[split_feature]
        if temp_value <= split_value:
            return cls.predict_line(model.left_tree, line, high=high+1)
        else:
            return cls.predict_line(model.right_tree, line, high=high+1)
       
    @classmethod
    def predict(cls, model, data):
        myre = []
        for line in data:
            te = cls.predict_line(model, line)
            myre.append(te)
        myre = np.array(myre).reshape(len(data), 1)
        return myre

In [469]:
data = load_iris()['data']
mylabel = load_iris()['target']
mylabel = np.array([[i] for i in mylabel])

In [470]:
"""
mymodel = mycarttree(4)
result = mymodel.fit(data, mylabel)
preds = mymodel.predict(result, data)
acc = np.sum(preds == mylabel) / 150
acc
"""

0.9866666666666667

In [476]:
np.random.permutation(np.arange(10))

array([8, 0, 9, 6, 1, 3, 5, 7, 4, 2])

In [478]:
import random

In [480]:
index = np.arange(10)
random.shuffle(index)
index

array([1, 7, 6, 5, 0, 9, 8, 4, 3, 2])

In [477]:
np.random.permutation(np.arange(10))

array([5, 7, 0, 1, 4, 3, 6, 8, 9, 2])

In [388]:
from multiprocessing import Process, Queue

In [481]:
def single_model(data, label, ratio=0.9):
    m, n = data.shape
    index = np.arange(m)
    random.shuffle(index)
    #index = np.random.permutation(np.arange(m))
    # feature_index = np.random.permutation(np.arange(n))
    nums = index[:int(ratio * m)]
    n_data = data[nums]
    n_label = label[nums]
    return n_data, n_label


def myjob(q, r=0.9, data=data, mylabel=mylabel):
    #r = np.random.uniform(0.8, 1)
    # 这种搞随机数是不可行的，
    my_data, my_label = single_model(data, mylabel, ratio=r)
    mymodel = mycarttree(2)
    result = mymodel.fit(my_data, my_label)
    
    m, _ = data.shape
    preds = mymodel.predict(result, data)
    acc = np.round(np.sum(preds == mylabel) / m, 2)
    print("this model's acc is {}".format(acc))
    q.put(preds)

In [483]:
q = Queue()
tmp1 = Process(target=myjob, args=(q, 0.9), name="job1")
tmp2 = Process(target=myjob, args=(q, 0.9), name="job2")
tmp3 = Process(target=myjob, args=(q, 0.9), name="job3")
tmp1.start()
tmp2.start()
tmp3.start()
tmp1.join()
tmp2.join()
tmp3.join()

this model's acc is 0.99
this model's acc is 0.97
this model's acc is 0.98


In [473]:
q = Queue()
myqueus = []

rlist=[0.7, 0.8, 0.85, 0.9, 0.95]
for i, r in zip(range(5), rlist):
    temp_job = Process(target=myjob, args=(q, r), name="jon_{}".format(i))
    myqueus.append(temp_job)
    
for jons in myqueus:
    jons.start()
    
for jons in myqueus:
    jons.join()
    
nps = -1 * np.ones((150, 1))
#for myres in q.get():
for _ in range(5):
    nps = np.concatenate([nps,q.get()], axis=1)
    #nps.append(q.get())

    
last_label = get_label(nps)
acc = np.round(np.sum(last_label == mylabel) / 150, 2)
print("last_label acc is {}".format(acc))

this model's acc is 0.96
this model's acc is 0.96
this model's acc is 0.99
this model's acc is 0.99
this model's acc is 0.99
last_label acc is 0.99


In [474]:
def get_label(nps_data=nps):
    last_label = []
    for line in nps_data:
        temp_label = Counter(line).most_common()[0][0]
        last_label.append([temp_label])
    return np.array(last_label)

# 二叉搜索树

In [266]:
class _node:
    def __init__(self, value=None, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right

class mysearch:            
    def __init__(self):
        self.mynode = _node()
    
    def insertdata(self, data):
        # 插入数据
        new_node = _node(value=data)
        if self.mynode.value is None:
            self.mynode = new_node
            return 
            
        temp = self.mynode
        if data > temp.value:
            if temp.right is None:
                temp.right = new_node
        else:
            if temp.left is None:
                temp.left = new_node
                
    @classmethod
    def print_nodevalue(cls, model,i=0):
        """
        如何递归的去打印子叶节点的，标号。#这里是中序遍历。
        """
        
        #if model.left is not None:
            #cls.print_nodevalue(model.left, i=i+1)
        if model.value is not None:
            # print(model.value)
            print("level {}, model value is {}".format(i, model.value))
        if model.left is not None:
            cls.print_nodevalue(model.left, i=i+1)
        if model.right is not None:
            cls.print_nodevalue(model.right, i=i+1)
            
    @classmethod
    def get_treedeep(cls, tree):
        """
        计算树的最大深度
        """
        max_deep = 0
        if tree.left is not None:
            temp_deep = 1 + cls.get_treedeep(tree.left)
            if temp_deep > max_deep:
                max_deep = temp_deep
        if tree.right is not None:
            temp_deep = 1 + cls.get_treedeep((tree.right))
            if temp_deep > max_deep:
                max_deep = temp_deep
        return max_deep
    
    
    @classmethod
    def find_value(cls, model, data):
        # 没法用 RETURN 
        if model.value == data:
            print("find")
        if model.left is not None:
            cls.find_value(model.left, data)
        if model.right is not None:
            cls.find_value(model.right, data)

    @property
    def allnode(self):
        return self.mynode

In [374]:
class newnode(mysearch):
    def __init__(self):
        super(newnode, self).__init__()
        self.mynode = _node()
    
    def insertdata(self, data):
        new_node = _node(value=data)
        if self.mynode.value is None:
            self.mynode = new_node
            return 
        
        # 实现多个输入的方法。temp, cur类似于指针的东西 在作用。
        temp = self.mynode
        while temp is not None:
            cur = temp
            if data > temp.value:
                temp = temp.right
            else:
                temp = temp.left
        
        # 给 指针插入新的数据点，就可以了。
        if data > cur.value:
            cur.right = new_node
        else:
            cur.left = new_node
        print("ok insertone")
        
    @classmethod
    def print_nodevalue(cls, model,i=0):
        """
        如何递归的去打印子叶节点的，标号。#这里是中序遍历。
        """
        #if model.left is not None:
            #cls.print_nodevalue(model.left, i=i+1)
        if model.left is not None:
            cls.print_nodevalue(model.left, i=i+1)
        if model.value is not None:
            # print(model.value)
            print("level {}, model value is {}".format(i, model.value))
        if model.right is not None:
            cls.print_nodevalue(model.right, i=i+1)
    
    @classmethod        
    def omit_node(cls, model, data):
        pass
    


In [375]:
my = s.allnode

In [376]:
s = newnode()
s.insertdata(9)
s.insertdata(1)
s.insertdata(4)
s.insertdata(0.9)
s.insertdata(10)
s.insertdata(25)
s.insertdata(9.5)
s.insertdata(3)
s.insertdata(2)
s.print_nodevalue(s.allnode)
s.get_treedeep(s.allnode)

ok insertone
ok insertone
ok insertone
ok insertone
ok insertone
ok insertone
ok insertone
ok insertone
level 2, model value is 0.9
level 1, model value is 1
level 4, model value is 2
level 3, model value is 3
level 2, model value is 4
level 0, model value is 9
level 2, model value is 9.5
level 1, model value is 10
level 2, model value is 25


4

In [377]:
s.find_value(s.allnode, 4)

find


In [634]:
# 
d = np.array([6, 3, 1, 4, 2, 5, 3, 7, 2])
d
d = np.array([2,3, 0, 1,0, 2, 5, 3, 3])
d = np.array([0, 1, 3, 2])

In [635]:
dicts = {}
for i in range(len(d)):
    #if i != d[i]:
    while i != d[i]:
        index = d[i]
        d[i], d[index] = d[index], index
        if d[i] == d[index]:
            print(d[i])
            dicts.setdefault(d[i], 0)
            dicts[d[i]] += 1
            break 

print(dicts)


{}


In [636]:
d

array([0, 1, 2, 3])